# Chapter 3: Coding Attention Mechanisms

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/06.webp" width="300px">

## 3.3 Attending to different parts of the input with self-attention

### 3.3.1 A simple self-attention mechanism without trainable weights



In [2]:
import torch

# for simplicity, each word = one token
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [3]:
input_query = inputs[1]
input_query

tensor([0.5500, 0.8700, 0.6600])

In [4]:
input_1 = inputs[0]
input_1

tensor([0.4300, 0.1500, 0.8900])

In [5]:
# manual visualization of dot product between [2] and [1]

0.55*0.43 + 0.87*0.15 + 0.66 * 0.89

0.9544

In [6]:
torch.dot(input_query, input_1)

tensor(0.9544)

In [7]:
res = torch.dot(inputs[1], input_query)
res

tensor(1.4950)

In [8]:
inputs.shape[0]

6

In [9]:
torch.empty(inputs.shape[0])

tensor([0., 0., 0., 0., 0., 0.])

In [10]:
query = inputs[1]  # 2nd input token is the query

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, input_query) # dot product (transpose not necessary here since they are 1-dim vectors)

# attention/similarity score is presented thru dot product of vectors in respect to x^2
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [11]:
# introductory method for normalization

attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

attn_weights_2_tmp

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])

In [12]:
attn_weights_2_tmp.sum()

tensor(1.0000)

In [13]:
# returns normalized tensor with optimized values for training

attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

In [14]:
# this is basically what happens in softmax function

def softmax_beta(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

softmax_beta(attn_scores_2)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

steps 1 (input vector calculation) and 2 (attention score normalization) are complete
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/10.webp" width="500px">

In [15]:
torch.zeros(query.shape)

tensor([0., 0., 0.])

In [16]:
query = inputs[1] # 2nd input token is the query for demo

context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    print(f"{attn_weights_2[i]} ---> {inputs[i]}")
    context_vec_2 += attn_weights_2[i] * inputs[i]

print(context_vec_2)

0.13854756951332092 ---> tensor([0.4300, 0.1500, 0.8900])
0.2378913015127182 ---> tensor([0.5500, 0.8700, 0.6600])
0.23327402770519257 ---> tensor([0.5700, 0.8500, 0.6400])
0.12399158626794815 ---> tensor([0.2200, 0.5800, 0.3300])
0.10818186402320862 ---> tensor([0.7700, 0.2500, 0.1000])
0.15811361372470856 ---> tensor([0.0500, 0.8000, 0.5500])
tensor([0.4419, 0.6515, 0.5683])


In [17]:
for i, x_i in enumerate(inputs):
    print(i, inputs[i])

0 tensor([0.4300, 0.1500, 0.8900])
1 tensor([0.5500, 0.8700, 0.6600])
2 tensor([0.5700, 0.8500, 0.6400])
3 tensor([0.2200, 0.5800, 0.3300])
4 tensor([0.7700, 0.2500, 0.1000])
5 tensor([0.0500, 0.8000, 0.5500])


### 3.3.2 Computing attention weights for all input tokens

In [19]:
# a more generalized version, however slower due to nested for loops

attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        # 6-6 matrix attention score
        attn_scores[i, j] = torch.dot(x_i, x_j) 

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [22]:
# more optimized version using matrix multiplication

attn_scores = inputs @ inputs.T
attn_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [27]:
# normalizes with respect to 1

attn_weights = torch.softmax(attn_scores, dim=1)
attn_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [28]:
torch.sum(torch.tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581]))

tensor(1.)

In [32]:
# automated version of all context vectors rather than only inputs[1]

all_context_vecs = attn_weights @ inputs
all_context_vecs

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

In [33]:
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


In [34]:
# summary of updated code

attn_scores = inputs @ inputs.T
attn_weights = torch.softmax(attn_scores, dim=1)
all_context_vecs = attn_weights @ inputs
all_context_vecs

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

## 3.4 Implementing self-attention with trainable weights

### 3.4.1 Computing the attention weights step by step
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/17.webp" width="600px">

In [35]:
x_2 = inputs[1] # second input element
d_in = inputs.shape[1] # the input embedding size, d=3
d_out = 2 # the output embedding size, d=2

In [38]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out))
W_key = torch.nn.Parameter(torch.rand(d_in, d_out))
W_value = torch.nn.Parameter(torch.rand(d_in, d_out))

In [39]:
# transforms 3-dimensional tensor into 2-dimensional

query_2 = x_2 @ W_query
query_2

tensor([0.4306, 1.4551], grad_fn=<SqueezeBackward4>)

In [43]:
keys = inputs @ W_key
value = inputs @ W_value

keys.shape

torch.Size([6, 2])

In [46]:
keys_2 = keys[1]
attn_score_22 = torch.dot(query_2, keys_2)

In [47]:
attn_scores_2 = query_2 @ keys.T
attn_scores_2

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
       grad_fn=<SqueezeBackward4>)

In [50]:
# step 3. obtain attention weights by normalizing scores

d_k = keys.shape[1]

attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
attn_weights_2

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
       grad_fn=<SoftmaxBackward0>)

In [52]:
attn_weights_2.sum()

tensor(1.0000, grad_fn=<SumBackward0>)

In [ ]:
# step 4. obtain context vector